<a href="https://colab.research.google.com/github/andreabochicchio02/MLP_in_CUDA/blob/main/MLP_sinc2D_tiled_mult.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Multi-Layer Perceptron (MLP) for REGRESSION**

Multi-Layer Perceptron to solve a regression problem (sinc2D function) implemented in CUDA using **GPU shared memory for multiplication**

**GPU su COLAB:** Runtime > Cambia tipo di runtime > GPU > Salva

### Including libraries and initializing MLP parameters

In [1]:
%%writefile MLP.cuh

#include <iostream>
#include <vector>
#include <array>
#include <cmath>
#include <algorithm>
#include <random>
#include <ctime>
#include <limits>

#include <cuda_runtime.h>
#include <device_launch_parameters.h>
#include <curand_kernel.h>

using namespace std;



const int num_train = 150*150;              // number of training pattern (put a square number here)
const int num_test = 2500;


// //////////////////////////////////////////// //
//                 MLP parameters               //
// //////////////////////////////////////////// //
const int n_output = 1;                     // Number of outputs
const int n_features = 2;                   // Number of input features
const int n_hidden = 300;                   // Number of neurons in the hidden layer
const int epochs = 500;                     // Number of epochs
const int minibatches = 30;                 // Number of mini-batches
__device__ float eta = 1e-6;                           // Learning rate


// WEIGHTS MODEL AND BIAS
array<array<float, n_features+1>, n_hidden> w1 = {};
array<array<float, n_hidden+1>, n_output> w2 = {};




// Global declaration of variable used in the train step
const int elem = (num_train + minibatches -1 )/minibatches;     // number of inputs used in each minibatch


// Threads kernel CUDA
const int tilex = 32;    // thread-block x
const int tiley = 32;    // thread-block y

// GPU DATA
float* dev_x_train;         // dataset train
float* dev_y_train;         // dataset train
float* dev_w1;              // pesi w1
float* dev_w2;              // pesi  w2
float* dev_w2_extend;       // column vector to matrix
int* dev_I;                 // matrice per indice dataset

//forward
float* dev_x_input;         // minibatch ingresso MLP
float* dev_y_input;         // minibatch ingresso MLP
float* dev_a0;              // input and bias
float* dev_rZ1;             // ris somma pesata
float* dev_rA1;             // uscita hidden layer
float* dev_a1;              // ingresso output layer
float* dev_rZ2;             // ris somma pesata output layer
float* dev_rA2;             // uscita MLP

float* dev_square_error;

//backpropagation
float* dev_dL_dZ2;
float* dev_dL_dZ2_extend;   // column vector to matrix
float* dev_dL_dW2;
float* dev_dL_dA1;
float* dev_sigma_prime;
float* dev_dL_drZ1;
float* dev_dL_dW1;

float* dev_delta_W1_unscaled;
float* dev_delta_W2_unscaled;

// predict
float* dev_x_pred;            // dataset train in predict (inizialization in predict)
float* dev_y_pred;            // dataset train in predict (inizialization in predict)

Writing MLP.cuh


### GPU allocation

In [2]:
%%writefile allocation.cu

void GPU_alloc(){
    cudaMalloc((void **)&dev_x_train, num_train * n_features * sizeof(float));
    cudaMalloc((void **)&dev_w1, n_hidden * (n_features+1) * sizeof(float));
    cudaMalloc((void **)&dev_w2, n_output * (n_hidden+1) * sizeof(float));
    cudaMalloc((void **)&dev_w2_extend, (n_hidden+1) * elem * sizeof(float));
    cudaMalloc((void **)&dev_y_train, num_train * n_output * sizeof(float));

    cudaMalloc((void **)&dev_I, minibatches * elem * sizeof(int));
    cudaMalloc((void **)&dev_x_input, elem * n_features * sizeof(float));
    cudaMalloc((void **)&dev_y_input, elem * n_output * sizeof(float));
    cudaMalloc((void **)&dev_a0, (n_features+1) * elem * sizeof(float));
    cudaMalloc((void **)&dev_rZ1, n_hidden * elem * sizeof(float));
    cudaMalloc((void **)&dev_rA1, n_hidden * elem * sizeof(float));
    cudaMalloc((void **)&dev_a1, (n_hidden+1) * elem * sizeof(float));
    cudaMalloc((void **)&dev_rZ2, n_output * elem * sizeof(float));
    cudaMalloc((void **)&dev_rA2, n_output * elem * sizeof(float));

    cudaMalloc((void **)&dev_square_error, elem * sizeof(float));

    cudaMalloc((void **)&dev_dL_dZ2, n_output * elem * sizeof(float));
    cudaMalloc((void **)&dev_dL_dZ2_extend, (n_hidden+1) * elem * sizeof(float));
    cudaMalloc((void **)&dev_dL_dW2, n_output * (n_hidden+1) * sizeof(float));
    cudaMalloc((void **)&dev_dL_dA1, (n_hidden+1) * elem * sizeof(float));
    cudaMalloc((void **)&dev_sigma_prime, n_hidden * elem * sizeof(float));
    cudaMalloc((void **)&dev_dL_drZ1, n_hidden * elem * sizeof(float));
    cudaMalloc((void **)&dev_dL_dW1, n_hidden * (n_features+1) * sizeof(float));

    cudaMalloc((void **)&dev_delta_W1_unscaled, n_hidden * (n_features+1) * sizeof(float));
    cudaMalloc((void **)&dev_delta_W2_unscaled, n_output * (n_hidden+1) * sizeof(float));
}


void GPU_free(){
    cudaFree(dev_x_train);
    cudaFree(dev_w1);
    cudaFree(dev_w2);
    cudaFree(dev_y_train);

    cudaFree(dev_I);
    cudaFree(dev_x_input);
    cudaFree(dev_y_input);
    cudaFree(dev_a0);
    cudaFree(dev_rZ1);
    cudaFree(dev_rA1);
    cudaFree(dev_a1);
    cudaFree(dev_rZ2);
    cudaFree(dev_rA2);

    cudaFree(dev_square_error);

    cudaFree(dev_dL_dZ2);
    cudaFree(dev_dL_dW2);
    cudaFree(dev_dL_dA1);
    cudaFree(dev_sigma_prime);
    cudaFree(dev_dL_drZ1);
    cudaFree(dev_dL_dW1);

    cudaFree(dev_delta_W1_unscaled);
    cudaFree(dev_delta_W2_unscaled);
}

Writing allocation.cu


### CUDA kernel

In [3]:
%%writefile kernel.cu

template <int TS> __global__ void tiled_mult(float * __restrict C, const float * __restrict A, const float * __restrict B,
                                             int rowA, int colA, int colB)
{
    __shared__ float Atile[TS][TS];         // tile in A
    __shared__ float Btile[TS][TS];         // tile in B

    int tx  = threadIdx.x;                  // tile col
    int ty  = threadIdx.y;                  // tile row
    int ocx = blockDim.x * blockIdx.x;      // tile x origin in C
    int ocy = blockDim.y * blockIdx.y;      // tile y origin in C

    int ay = ocy + ty;          // i in first tile on A and C
    int bx = ocx + tx;          // j in first tile on B and C

    float csum = 0.0f;

    int numTiles = (colA + TS - 1) / TS;

    #pragma unroll
    for(int t = 0; t < numTiles; t++) {
        int ax = t * TS + tx;    // current column in A
        int by = t * TS + ty;    // current row in B


        if (ay < rowA && ax < colA)
            Atile[ty][tx] = A[ay*colA + ax];    // copy A tile to shared mem
        else
            Atile[ty][tx] = 0.0f;


        if (by < colA && bx < colB)
            Btile[ty][tx] = B[by*colB + bx];    // copy B tile to shared mem
        else
            Btile[ty][tx] = 0.0f;


        __syncthreads();

        for (int k = 0; k<TS; k++) {
                csum += Atile[ty][k] * Btile[k][tx];
        }

        __syncthreads();

        ax += TS;               // step A tiles along rows of A
        by += TS;               // step B tiles down cols of B
    }

    // store complete result
    if (ay < rowA && bx < colB)
        C[ay * colB + bx] = csum;
}



template <int TS> __global__ void tiled_mult_transposeB(float * __restrict C,
 	    const float * __restrict A, const float * __restrict B, int rowA, int colA, int rowB)
{
	__shared__ float Atile[TS][TS];  // tile in A
	__shared__ float Btile[TS][TS];  // tile in B

	int tx  = threadIdx.x;            // tile col
	int ty  = threadIdx.y;            // tile row
	int ocx = blockDim.x*blockIdx.x;  // tile x origin in C
	int ocy = blockDim.y*blockIdx.y;  // tile y origin in C


	int ay = ocy + ty;      // i in first tile on A and C
	int bx = ocx + tx;      // j in first tile on B

	float csum = 0.0f;

    int numTiles = (colA + TS - 1) / TS;

    #pragma unroll
    for(int t = 0; t < numTiles; t++) {
        int ax = t * TS + tx;    // current column in A
        int by = t * TS + ty;    // current row in B

        if (ay < rowA && ax < colA)
		    Atile[ty][tx] = A[ay*colA + ax];  // copy A tile to shared mem
        else
            Atile[ty][tx] = 0.0f;


        if (by < colA && bx < rowB)
		    Btile[ty][tx] = B[bx * colA + by];  // copy B tile to shared mem
        else
            Btile[ty][tx] = 0.0f;


		__syncthreads();
		for(int k=0; k<TS; k++){
            csum += Atile[ty][k]*Btile[k][tx];
        }
		__syncthreads();

		ax += TS;         // step A tiles along rows of A
		by += TS;         // step B tiles along rows of B
	}

    // store complete result
    if (ay < rowA && bx < rowB)
	    C[ay*rowB + bx] = csum;
}

template <int TS> __global__ void tiled_mult_transposeA(float * __restrict C, const float * __restrict A, const float * __restrict B,
                                             int rowA, int colA, int colB)
{
    __shared__ float Atile[TS][TS];         // tile in A
    __shared__ float Btile[TS][TS];         // tile in B

    int tx  = threadIdx.x;                  // tile col
    int ty  = threadIdx.y;                  // tile row
    int ocx = blockDim.x * blockIdx.x;      // tile x origin in C
    int ocy = blockDim.y * blockIdx.y;      // tile y origin in C

    int ay = ocy + ty;          // i in first tile on A and C
    int bx = ocx + tx;          // j in first tile on B and C

    float csum = 0.0f;

    int numTiles = (rowA + TS - 1) / TS;

    #pragma unroll
        for(int t = 0; t < numTiles; t++) {
        int ax = t * TS + tx;    // current column in A
        int by = t * TS + ty;    // current row in B

        if (ay < colA && ax < rowA)
            Atile[ty][tx] = A[ax * colA + ay];    // copy A tile to shared mem
        else
            Atile[ty][tx] = 0.0f;


        if (by < rowA && bx < colB)
            Btile[ty][tx] = B[by * colB + bx];    // copy B tile to shared mem
        else
            Btile[ty][tx] = 0.0f;

        __syncthreads();

        for (int k = 0; k < TS; k++) {
            csum += Atile[ty][k] * Btile[k][tx];
        }
        __syncthreads();

        ax += TS;               // step A tiles along rows of A
        by += TS;               // step B tiles down cols of B
    }

    // store complete result
    if (ay < colA && bx < colB)
        C[ay * colB + bx] = csum;
}


__global__ void setMinibatchIdx(int* dev_I, int minibatches, int elem) {
    int idx_x = blockIdx.x * blockDim.x + threadIdx.x;
    int idx_y = blockIdx.y * blockDim.y + threadIdx.y;


    if (idx_x < elem && idx_y < minibatches){
        int i = idx_y * elem + idx_x;

        int row = i % minibatches;
        int col = i / minibatches;

        dev_I[row * elem + col] = i;                // set index
    }
}

__global__ void setBatch(float* x, float* x_input, int row, int col, int m) {
    int tid_x = threadIdx.x + blockIdx.x * blockDim.x;
    int tid_y = threadIdx.y + blockIdx.y * blockDim.y;

    if (tid_x < col && tid_y < row) {
        int idx = tid_y + (m - 1) * row;
        int x_idx = idx * col + tid_x;
        int x_input_idx = tid_y * col + tid_x;

        x_input[x_input_idx] = x[x_idx];      // Batch copy of data for MLP input
    }
}


// Transpose matrix and set first row to 1
__global__ void transpose_extend(float* input, float* output, int row, int col) {
    int idx_x = blockIdx.x * blockDim.x + threadIdx.x;
    int idx_y = blockIdx.y * blockDim.y + threadIdx.y;

    if (idx_y < row && idx_x <= col) {
        output[idx_x * row + idx_y] = (idx_x == 0) ? 1.0f : input[idx_y * col + (idx_x-1)];
    }
}


__global__ void func_sigmoid(float* output, const float* input, int row, int col) {
    int idx_x = blockIdx.x * blockDim.x + threadIdx.x;
    int idx_y = blockIdx.y * blockDim.y + threadIdx.y;

    if (idx_y < row && idx_x < col) {
        int idx = idx_y * col + idx_x;

        output[idx] = 1.0f / (1.0f + expf(-input[idx]));
    }
}

__global__ void sigmoid_gradient(float* output, const float* rA1, int row, int col) {
    int idx_x = blockIdx.x * blockDim.x + threadIdx.x;
    int idx_y = blockIdx.y * blockDim.y + threadIdx.y;

    if (idx_y < row && idx_x < col) {
        int idx = idx_y * col + idx_x;

        output[idx] = rA1[idx] * (1.0 - rA1[idx]);
    }
}

__global__ void extend_1(float* output, float* input, int row, int col) {
    int idx_x = blockIdx.x * blockDim.x + threadIdx.x;
    int idx_y = blockIdx.y * blockDim.y + threadIdx.y;

    if (idx_y < row && idx_x <= col) {
        output[idx_y * col + idx_x] = (idx_y == 0) ? 1.0f : input[(idx_y - 1) * col + idx_x];
    }
}

__global__ void elem_mult_elem(float* A, const float* B, float* C, int row, int col) {
    int idx_x = blockIdx.x * blockDim.x + threadIdx.x;
    int idx_y = blockIdx.y * blockDim.y + threadIdx.y;

    if (idx_y < row && idx_x < col) {
        int idx = idx_y * col + idx_x;

        C[idx] = A[idx] * B[idx];
    }
}

__global__ void elem_sub(float* A, const float* B, float* C, int row, int col) {
    int idx_x = blockIdx.x * blockDim.x + threadIdx.x;
    int idx_y = blockIdx.y * blockDim.y + threadIdx.y;

    if (idx_y < row && idx_x < col) {
        int idx = idx_y * col + idx_x;

        C[idx] = A[idx] - B[idx];
    }
}

__global__ void copy(const float* input, float* output, int row, int col) {
    int idx_x = blockIdx.x * blockDim.x + threadIdx.x;
    int idx_y = blockIdx.y * blockDim.y + threadIdx.y;

    if (idx_y < row && idx_x < col) {
        int idx = idx_y * col + idx_x;

        output[idx] = input[idx];
    }
}

__global__ void update_weights(const float* input, float* output, int row, int col) {
    int idx_x = blockIdx.x * blockDim.x + threadIdx.x;
    int idx_y = blockIdx.y * blockDim.y + threadIdx.y;

    if (idx_y < row && idx_x < col) {
        int idx = idx_y * col + idx_x;

        output[idx] -= eta * input[idx];
    }
}

__global__ void square_error(float* A, const float* B, float* C, int n_hidden, int elem) {
    int idx_x = blockIdx.x * blockDim.x + threadIdx.x;
    int idx_y = blockIdx.y * blockDim.y + threadIdx.y;

    int idx = idx_y * elem + idx_x;

    if (idx_y < n_hidden && idx_x < elem) {
        float diff = A[idx] - B[idx];
        C[idx] = diff * diff;
    }
}

__global__ void initialize_weights(float *W, curandState *states, long seed, int rows, int cols) {
    int idx_y = blockIdx.y * blockDim.y + threadIdx.y;
    int idx_x = blockIdx.x * blockDim.x + threadIdx.x;

    int idx = idx_y * cols + idx_x;

    if (idx_y < rows && idx_x < cols) {
        curand_init(seed+idx, 0, 0, &states[idx]);

        curandState localState = states[idx];
        W[idx] = 2.0f * curand_uniform(&localState) - 1.0f;
        states[idx] = localState;
    }
}


__global__ void reduce(float *y, float *x, int N){
    extern __shared__ float tsum[];

    int id = threadIdx.x;
    int tid = blockDim.x*blockIdx.x+threadIdx.x;
    int stride =  gridDim.x*blockDim.x;

    tsum[id] = 0.0f;
    for(int k=tid;k<N;k+=stride)
        tsum[id] += x[k];

    __syncthreads();

    // number of blocks power of 2 greater than blockDim
    int block2 = 1;
    while (block2 < blockDim.x) block2 *= 2;

    // power of 2 reduction loop
    for(int k=block2/2; k>0; k >>= 1){
        if(id<k && id+k < blockDim.x)
            tsum[id] += tsum[id+k];
        __syncthreads();
    }

    if(id==0) y[blockIdx.x] = tsum[0];
}

__global__ void transpose_extend_row_vector(const float* input, float* output, int row, int col) {
    int idx_x = blockIdx.x * blockDim.x + threadIdx.x;
    int idx_y = blockIdx.y * blockDim.y + threadIdx.y;

    if (idx_y < row && idx_x < col) {
        int idx = idx_y * col + idx_x;
        output[idx] = input[idx_y];
    }
}

__global__ void extend_row_vector(const float* input, float* output, int row, int col) {
    int idx_x = blockIdx.x * blockDim.x + threadIdx.x;
    int idx_y = blockIdx.y * blockDim.y + threadIdx.y;

    // Calcola l'indice lineare dell'elemento nella matrice
    if (idx_y < row && idx_x < col) {
        int idx = idx_y * col + idx_x;  // Accesso in riga (row-major)
        output[idx] = input[idx_x];
    }
}

Writing kernel.cu


###Math function

In [4]:
%%writefile math_function.cu
#include "kernel.cu"

void A_mult_B(const float* A, const float* B, float* C,
              int rowA, int colA, int colB) {

    dim3 threads ={tilex,tiley,1};
    dim3 blocks ={(colB+threads.x-1)/threads.x,(rowA+threads.y-1)/threads.y,1};

    tiled_mult<tilex><<<blocks,threads>>>(C, A, B, rowA, colA, colB);
}


void A_mult_B_T(const float* A, const float* B, float* C,
                   int rowA, int colA, int rowB) {

    dim3 threads ={tilex,tiley,1};
    dim3 blocks ={(rowB+threads.x-1)/threads.x,(rowA+threads.y-1)/threads.y,1};

    tiled_mult_transposeB<tilex><<<blocks,threads>>>(C, A, B, rowA, colA, rowB);
}

void A_T_mult_B(const float* A, const float* B, float* C,
                int rowA, int colA, int colB) {

    dim3 threads ={tilex,tiley,1};
    dim3 blocks ={(colB+threads.x-1)/threads.x,(colA+threads.y-1)/threads.y,1};

    tiled_mult_transposeA<tilex><<<blocks,threads>>>(C, A, B, rowA, colA, colB);
}

Writing math_function.cu


### Initialization
Initialize train and test input and output

In [5]:
%%writefile gen.cu

//sinc2D function generation
void sinc2D_gen(float* x, float* y, int num_patterns){
    int num_points = sqrt(num_patterns);

    // linspace x1
    vector<float> x1(num_points);
    float start_x1 = -5.0;
    float end_x1 = 5.0;
    float step_x1 = (end_x1 - start_x1) / (num_points - 1);
    for (int i = 0; i < num_points; ++i){
        x1[i] = start_x1 + i * step_x1;
    }


    // linspace x2
    vector<float> x2(num_points);
    float start_x2 = -5.0;
    float end_x2 = 5.0;
    float step_x2 = (end_x2 - start_x2) / (num_points - 1);
    for (int i = 0; i < num_points; ++i){
        x2[i] = start_x2 + i * step_x2;
    }


    // meshgrid
    vector<vector<float>> XX1(num_points, vector<float>(num_points));
    vector<vector<float>> XX2(num_points, vector<float>(num_points));
    for (int i = 0; i < num_points; ++i){
        for (int j = 0; j < num_points; ++j){
            XX1[i][j] = x1[j];
            XX2[i][j] = x2[i];
        }
    }


    // sinc2D
    vector<vector<float>> YY(num_points, vector<float>(num_points));
    for (int i = 0; i < num_points; ++i) {
        for (int j = 0; j < num_points; ++j) {
            float sinc_x1 = (XX1[i][j] == 0) ? 1.0 : sin(XX1[i][j]) / XX1[i][j];
            float sinc_x2 = (XX2[i][j] == 0) ? 1.0 : sin(XX2[i][j]) / XX2[i][j];
            YY[i][j] = 10.0 * sinc_x1 * sinc_x2;
        }
    }


    // initialization x e y
    for (int i = 0; i < num_points; ++i) {
        for (int j = 0; j < num_points; ++j) {
            x[(i*num_points+j)*n_features] = XX1[j][i];
            x[(i*num_points+j)*n_features + 1] = XX2[j][i];
            y[i * num_points + j] = YY[j][i];
        }
    }
}

Writing gen.cu


###Forward function
Compute the forward step

In [6]:
%%writefile forward.cu

// Compute the forward step
void MLP_MSELIN_forward(int elem){
    // rA0: is the "reduced A0" and it coincides with the transpose of the tall input matrix (nObs x nInput)
    // A0  = E(rA0) It is the "extended" version of rA0, obtained by it by adding a row of ones as its new first row
    // Extend matrix X by adding the bias
    dim3 threads ={tilex,tiley,1};
    dim3 blocks ={(n_features+threads.x-1)/threads.x, (elem+threads.y-1)/threads.y, 1};
    transpose_extend<<<blocks, threads>>>(dev_x_input, dev_a0, elem, n_features);


    // rZ1 = \sum(W1,A0).  It is the pre-activation at layer 1 (the hidden one)
    A_mult_B(dev_w1, dev_a0, dev_rZ1, n_hidden, n_features+1, elem);


    // rA1 = \sigma(rZ1).  It is the output of the first layer (the hidden one)
    blocks ={(elem+threads.x-1)/threads.x, (n_hidden+threads.y-1)/threads.y, 1};
    func_sigmoid<<<blocks, threads>>>(dev_rA1, dev_rZ1, n_hidden, elem);


    // MLP_extend
    // A1  = E(rA1).       It is the extended version of rA1
    // Extend matrix X by adding the bias
    blocks ={(elem+threads.x-1)/threads.x, ((n_hidden+1)+threads.y-1)/threads.y, 1};
    extend_1<<<blocks, threads>>>(dev_a1, dev_rA1, (n_hidden+1), elem);


    // rZ2 = \sum(W2,A1).  It is the pre-activation at layer 2 (the output one)
    A_mult_B(dev_w2, dev_a1, dev_rZ2, n_output, n_hidden+1, elem);

    dev_rA2 = dev_rZ2;
}

Writing forward.cu


### Train
Learn weights from training data

In [7]:
%%writefile train.cu
#include "math_function.cu"
#include "forward.cu"
#include "allocation.cu"

float MLP_MSE_cost(int elem) {
    // Calculate the differences between y and rA2
    dim3 threads = {tilex, 1, 1};
    dim3 blocks ={(elem + threads.x-1)/threads.x,1,1};
    square_error<<<blocks, threads>>>(dev_rA2, dev_y_input, dev_square_error, n_output, elem);


    float* dev_temp_sum;        //partial sum of block
    cudaMalloc((void **)&dev_temp_sum, blocks.x * sizeof(float));

    //partial sum for each block
    reduce<<<blocks,threads, threads.x*sizeof(float)>>>(dev_temp_sum, dev_square_error, elem);      //somma parziale ogni blocco

    int blocchi = blocks.x;
    reduce<<<1, blocks.x, blocks.x * sizeof(float)>>>(dev_square_error, dev_temp_sum, blocchi);

    float res;
    cudaMemcpy(&res, &dev_square_error[0], sizeof(float), cudaMemcpyDeviceToHost);

    cudaFree(dev_temp_sum);

    return res;
}



// Compute the partial derivative of the loss with respect to the two weighting matrices W2 and W1, using the backpropagation algorithm.
void MLP_MSELIN_backprop(){
    // rA2 is 1xB
    // A1  is (H+1)xB
    // A0  is (D+1)xB
    // rZ1 is HxB
    // y   is 1xB
    // W1  is Hx(D+1)
    // W2  is 1x(H+1)

    // Step 1: compute dL_dZ2 of size 1xB
    // NB: rA2 coincides with y_pred
    // NB: dL_dZe could be called "grad2", the gradient on the output layer with respect the pre-activation Z2

    dim3 threads = {tilex, 1, 1};
    dim3 blocks ={(elem + threads.x-1)/threads.x,1,1};
    elem_sub<<<blocks, threads>>>(dev_rA2, dev_y_input, dev_dL_dZ2, n_output, elem);


    // Step 2: compute dL_dW2 % size 1x(H+1)
    // NB: dL_dW2 could be called "delta_W2_unscaled", because it is of the same size of W2 and stores the unscaled variation
    A_mult_B_T(dev_dL_dZ2, dev_a1, dev_dL_dW2, n_output, elem, n_hidden+1);

    // Step 3: compute dL_dA1 of size (H+1)xB
    threads = {tilex, tiley, 1};
    blocks ={(elem + threads.x-1)/threads.x,((n_hidden+1)+threads.y-1)/threads.y,1};
    transpose_extend_row_vector<<<blocks, threads>>>(dev_w2, dev_w2_extend, (n_hidden+1), elem);
    extend_row_vector<<<blocks, threads>>>(dev_dL_dZ2, dev_dL_dZ2_extend, (n_hidden+1), elem);
    elem_mult_elem<<<blocks, threads>>>(dev_w2_extend, dev_dL_dZ2_extend, dev_dL_dA1, (n_hidden+1), elem);



    // Step 4: compute dL_drZ1 of size HxB (also sigma_prime_of_rZ1 has size HxB)
    // NB: dL_drZ1 could have been called "grad1", since it is the gradient at the first layer (the hidden one), with respect to Z1
    threads = {tilex, tiley, 1};
    blocks ={(elem + threads.x-1)/threads.x,(n_hidden+threads.y-1)/threads.y,1};
    sigmoid_gradient<<<blocks, threads>>>(dev_sigma_prime, dev_rA1, n_hidden, elem);
    elem_mult_elem<<<blocks, threads>>>(&dev_dL_dA1[elem], dev_sigma_prime, dev_dL_drZ1, n_hidden, elem);



    // Step 5: compute dL_dW1 of size Hx(D+1)
    // NB: dL_dW1 could be called "delta_W1_unscaled", because it is of the same size of W2 and stores the unscaled variation of W1
    A_mult_B_T(dev_dL_drZ1, dev_a0, dev_dL_dW1, n_hidden, elem, n_features+1);


    blocks ={((n_features+1) + threads.x-1)/threads.x,(n_hidden+threads.y-1)/threads.y,1};
    copy<<<blocks, threads>>>(dev_dL_dW1, dev_delta_W1_unscaled, n_hidden, (n_features+1));


    blocks ={((n_hidden+1) + threads.x-1)/threads.x,(n_output+threads.y-1)/threads.y,1};
    copy<<<blocks, threads>>>(dev_dL_dW2, dev_delta_W2_unscaled, n_output, (n_hidden+1));


    /* -----------------------------------------------------------------------------
    NB: grad2 is the gradient at the hidden layer.
    It is a column vector in the case of a single pattern
    (minibatch equal to the training set site) or a matrix,
    to be imagined, in the latter case, a matrix of columns,
    the gradients of each input pattern in the minibatch.

    NB: grad1 is the gradient at the hidden layer (derivative
    of the loss with respect Z1, the pre-activation at the hidden layer).
    It is a column vector in the case of a single pattern
    (minibatch equal to the training set site) or a matrix,
    to be imagined, in the latter case, a matrix of columns,
    the gradients of each input pattern in the minibatch.
    ----------------------------------------------------------------------------- */
}


// learn weights from training data
void MLP_MSELIN_train(const array<array<float, n_features>, num_train> &x, const array<float, num_train> &y){
    GPU_alloc();

    // initialize weights w1 and w2
    random_device rd;
    long seed = rd();

    curandState *states_w1;         //required for cuRAND
    cudaMalloc(&states_w1, n_hidden * (n_features+1) * sizeof(curandState));

    dim3 threads ={tilex,tiley,1};
    dim3 blocks ={((n_features+1) + threads.x-1)/threads.x, (n_hidden+threads.y-1)/threads.y,1};
    initialize_weights<<<blocks, threads>>>(dev_w1, states_w1, seed, n_hidden, (n_features+1));

    curandState *states_w2;         //required for cuRAND
    cudaMalloc(&states_w2, n_output * (n_hidden+1) * sizeof(curandState));

    blocks ={((n_hidden+1) + threads.x-1)/threads.x, (n_output+threads.y-1)/threads.y,1};
    initialize_weights<<<blocks, threads>>>(dev_w2, states_w2, seed, n_output, (n_hidden+1));

    cudaFree(states_w1);
    cudaFree(states_w2);



    // COPIA DATASET GPU
    cudaMemcpy(dev_x_train, &x[0][0], num_train * n_features * sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpy(dev_y_train, &y[0], num_train * n_output * sizeof(float), cudaMemcpyHostToDevice);


    blocks ={(elem + threads.x-1)/threads.x,(minibatches+threads.y-1)/threads.y,1};
    setMinibatchIdx<<<blocks, threads>>>(dev_I, minibatches, elem);

    // loop: epochs
    for(int e=1; e<=epochs; ++e) {

        // loop: minibatches
        for(int m=1; m<=minibatches; ++m){

            // input for this minibatch
            blocks ={(n_features + threads.x-1)/threads.x,(elem+threads.y-1)/threads.y,1};
            setBatch<<<blocks, threads>>>(dev_x_train, dev_x_input, elem, n_features, m);

            // Feedforward
            MLP_MSELIN_forward(elem);


            // output for this minibatch
            threads ={1,tiley,1};
            blocks ={(n_output + threads.x-1)/threads.x,(elem+threads.y-1)/threads.y,1};
            setBatch<<<blocks, threads>>>(dev_y_train, dev_y_input, elem, n_output, m);

            float res = MLP_MSE_cost(elem);
            float step_cost = res / (2.0f * elem);
            printf("Epoch %d/%d, minibatch %04d, Loss (MSE) %g\n", e, epochs, m, step_cost);

            // Compute gradient via backpropagation
            MLP_MSELIN_backprop();

            threads ={tilex,tiley,1};
            blocks ={((n_features+1) + threads.x-1)/threads.x,(n_hidden + threads.y-1)/threads.y,1};
            update_weights<<<blocks, threads>>>(dev_delta_W1_unscaled, dev_w1, n_hidden , (n_features+1));

            blocks ={((n_hidden+1) + threads.x-1)/threads.x,(n_output+ threads.y-1)/threads.y,1};
            update_weights<<<blocks, threads>>>(dev_delta_W2_unscaled, dev_w2, n_output, (n_hidden+1));
        }

    }

    cudaMemcpy(&w1[0][0], dev_w1, n_hidden * (n_features+1) * sizeof(float), cudaMemcpyDeviceToHost);
    cudaMemcpy(&w2[0][0], dev_w2, n_output * (n_hidden+1) * sizeof(float), cudaMemcpyDeviceToHost);
}

Writing train.cu


###Predict

In [8]:
%%writefile predict.cu

// Predict the outputs for all the observations in X, where each row of X is a distinct observation.
float MLP_MSELIN_predict(float* x, float* y, int tot_elem) {
    float sum = 0;

    cudaMalloc((void **)&dev_x_pred, tot_elem * n_features * sizeof(float));
    cudaMemcpy(dev_x_pred, x, tot_elem * n_features * sizeof(float), cudaMemcpyHostToDevice);

    cudaMalloc((void **)&dev_y_pred,  tot_elem * n_output * sizeof(float));
    cudaMemcpy(dev_y_pred, y, tot_elem * n_output * sizeof(float), cudaMemcpyHostToDevice);


    for (int m = 1; m * elem <= tot_elem+(elem-1); ++m) {   // precautions in case tot_elem is not a multiple of elem
        int current_elem = (m * elem <= tot_elem) ? elem : (tot_elem - (m - 1) * elem);

        dim3 threads ={tilex,tiley,1};
        dim3 blocks ={(n_features + threads.x-1)/threads.x,(current_elem+threads.y-1)/threads.y,1};
        setBatch<<<blocks, threads>>>(dev_x_pred, dev_x_input, current_elem, n_features, m);

        blocks ={(n_output + threads.x-1)/threads.x,(current_elem+threads.y-1)/threads.y,1};
        setBatch<<<blocks, threads>>>(dev_y_pred, dev_y_input, current_elem, n_output, m);

        MLP_MSELIN_forward(current_elem);

        sum += MLP_MSE_cost(current_elem);
    }

    float step_cost = sum / (2.0f * tot_elem);

    cudaFree(dev_x_pred);
    cudaFree(dev_y_pred);

    return step_cost;
}

Writing predict.cu


###Main

In [9]:
%%writefile main.cu
#include "MLP.cuh"
#include "gen.cu"
#include "train.cu"
#include "predict.cu"

int main() {
    array<array<float, n_features>, num_train> x_train;
    array<float, num_train> y_train;
    sinc2D_gen(x_train[0].data(), y_train.data(), num_train);

    array<array<float, n_features>, num_test> x_test;
    array<float, num_test> y_test;
    sinc2D_gen(x_test[0].data(), y_test.data(), num_test);



    // Shuffling training data
    array<int, num_train> shuffled_ind;
    for (int i = 0; i < num_train; ++i) {
        shuffled_ind[i] = i;
    }

    default_random_engine generator(std::time(nullptr));
    shuffle(shuffled_ind.begin(), shuffled_ind.end(), generator);

    array<array<float, n_features>, num_train> x_train_temp;
    array<float, num_train> y_train_temp;

    for (int i = 0; i < num_train; ++i) {
        x_train_temp[i] = x_train[shuffled_ind[i]];
        y_train_temp[i] = y_train[shuffled_ind[i]];
    }

    x_train = x_train_temp;
    y_train = y_train_temp;

    // Learn weights from training data
    MLP_MSELIN_train(x_train, y_train);

    // Predict the outputs for all the observations in X
    float acc_train = MLP_MSELIN_predict(x_train[0].data(), y_train.data(), num_train);
    printf("Training accuracy (MSE): %g\n", acc_train);

    float acc_test = MLP_MSELIN_predict(x_test[0].data(), y_test.data(), num_test);
    printf("Test accuracy: (MSE): %g\n", acc_test);

    GPU_free();


    return 0;
}

Writing main.cu


###Run

In [ ]:
!rm prog
!nvcc -o prog main.cu

In [ ]:
!./prog